# 8.1 OOP and Inheritance: "Is-a" Relationships

In [1]:
# File employees.py
from __future__ import print_function
class Employee:
    def __init__(self, name, salary=0):
        self.name = name
        self.salary = salary
    def giveRaise(self, percent):
        self.salary = self.salary + (self.salary * percent)
    def work(self):
        print(self.name, "does stuff")
    def __repr__(self):
        return "<Employee: name=%s, salary=%s>" % (self.name, self.salary)
class Chef(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 50000)
    def work(self):
        print(self.name, "makes food")
class Server(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 40000)
    def work(self):
        print(self.name, "interfaces with customer")
class PizzaRobot(Chef):
    def __init__(self, name):
        Chef.__init__(self, name)
    def work(self):
        print(self.name, "makes pizza")
if __name__ == "__main__":
    bob = PizzaRobot('bob') # Make a robot named bob
    print(bob) # Run inherited __repr__
    bob.work() # Run type-specific action
    bob.giveRaise(0.20) # Give bob a 20% raise
    print(bob); print()
    for klass in Employee, Chef, Server, PizzaRobot:
        obj = klass(klass.__name__)
        obj.work()

<Employee: name=bob, salary=50000>
bob makes pizza
<Employee: name=bob, salary=60000.0>

Employee does stuff
Chef makes food
Server interfaces with customer
PizzaRobot makes pizza


## 8.2 OOP and Composition: "Has a" Relationships

In [2]:
# File pizzashop.py
from __future__ import print_function
from employees import PizzaRobot, Server
class Customer:
    def __init__(self, name):
        self.name = name
    def order(self, server):
        print(self.name, "orders from", server)
    def pay(self, server):
        print(self.name, "pays for item to", server)
class Oven:
    def bake(self):
        print("oven bakes")
class PizzaShop:
    def __init__(self):
        self.server = Server('Pat') # Embed other objects
        self.chef = PizzaRobot('Bob') # A robot named bob
        self.oven = Oven()
    def order(self, name):
        customer = Customer(name) # Activate other objects
        customer.order(self.server) # Customer orders from server
        self.oven.bake()
        customer.pay(self.server)
if __name__ == "__main__":
    scene = PizzaShop() # Make the composite
    scene.order('Homer') # Simulate Homer's order
    print('...')
    scene.order('Shaggy') # Simulate Shaggy's orde

Homer orders from <Employee: name=Pat, salary=40000>
oven bakes
Homer pays for item to <Employee: name=Pat, salary=40000>
...
Shaggy orders from <Employee: name=Pat, salary=40000>
oven bakes
Shaggy pays for item to <Employee: name=Pat, salary=40000>


In [3]:
# File streams.py
def processor(reader, converter, writer):
    while True:
        data = reader.read()
        if not data: break
        data = converter(data)
        writer.write(data)
class Processor:
    def __init__(self, reader, writer):
        self.reader = reader
        self.writer = writer
    def process(self):
        while True:
            data = self.reader.readline()
            if not data: break
            data = self.converter(data)
            self.writer.write(data)
    def converter(self, data):
        assert False, 'converter must be defined'

# 8.3 OOP and Delegation: "Wrapper" Objects

In [4]:
# File trace.py
class Wrapper:
    def __init__(self, object):
        self.wrapped = object # Save object
    def __getattr__(self, attrname):
        print('Trace: ' + attrname) # Trace fetch
        return getattr(self.wrapped, attrname) # Delegate fetch

In [5]:
x = Wrapper([1, 2, 3])
x.append(4)

Trace: append


In [6]:
x.wrapped

[1, 2, 3, 4]

In [7]:
x=Wrapper({'a':1,'b':2})
list(x.keys())

Trace: keys


['a', 'b']

# 8.4 Pseudoprivate Attributes

In [8]:
class C1:       # Định nghĩa lớp bình thường
    def meth1(self): self.X = 88 
    def meth2(self): print(self.X)
class C2:
    def metha(self): self.X = 99 
    def methb(self): print(self.X)

In [9]:
class C1:
    def meth1(self): self.__X = 88 
    def meth2(self): print(self.__X) # Becomes _C1__X in I
class C2:
    def metha(self): self.__X = 99 
    def methb(self): print(self.__X) # Becomes _C2__X in I
class C3(C1, C2): pass

In [10]:
I = C3()
I.meth1(); I.metha()

In [11]:
print(I.__dict__)
I.meth2(); I.methb()

{'_C1__X': 88, '_C2__X': 99}
88
99


In [12]:
class Person:
    name="Lambda"  # Attribute class access
    _age=21
    __gender="Male"
    def __init__(self,name,age,gender):
        self.name=name  # Attribute instance access
        self._age=age
        self.__gender=gender
    def public(self):   # Method instance access
        pass
    def _protected(self):
        pass
    def __private(self):
        pass

# 8.5 Bound and Unbound methods

In [13]:
class Spam:
    def doit(self, message):
        print(message)
object1 = Spam()
object1.doit('hello world')

hello world


In [14]:
object1 = Spam()
object1.doit

<bound method Spam.doit of <__main__.Spam object at 0x034059D0>>

In [15]:
object1 = Spam()
x = object1.doit # Bound method object: instance+function
x('hello world')  # instance.method

hello world


In [16]:
object1 = Spam()
t = Spam.doit # Unbound method object (a function in 3.X: see ahead)
t(object1, 'howdy') # class.method(instance,args,.....)

howdy


In [17]:
class Eggs:
    def m1(self, n):
        print(n)
    def m2(self):
        x = self.m1 # Another bound method object
        x(42) # Looks like a simple function
Eggs().m2() # Prints 42

42


In [18]:
class Selfless:
    def __init__(self, data):
        self.data = data
    def selfless(arg1, arg2): # A simple function in 3.X
        return arg1 + arg2
    def normal(self, arg1, arg2): # Instance expected when called
        return self.data + arg1 + arg2

In [19]:
X = Selfless(2)
X.normal(3, 4) # instance.method

9

In [20]:
Selfless.normal(X, 3, 4) # class.method

9

In [21]:
Selfless.selfless(3, 4)  # class.method

7

Do trong phương thức selfless() không có đối số self nên sẽ gây lỗi.
```python
X.selfless(3, 4)  # instance.method
```
`Output: TypeError: selfless() takes 2 positional arguments but 3 were given`

Không truyền đủ đối số nên cũng gây ra lỗi.
```python
Selfless.normal(3, 4) # class.method
```
`Output: TypeError: normal() missing 1 required positional argument: 'arg2'`

In [22]:
class Number:
    def __init__(self, base):
        self.base = base
    def double(self):
        return self.base * 2
    def triple(self):
        return self.base * 3

In [23]:
x = Number(2) # Class instance objects
y = Number(3) # State + methods
z = Number(4)
x.double()

4

In [24]:
acts = [x.double, y.double, y.triple, z.double] # List of bound methods
for act in acts: # Calls are deferred
    print(act())

4
6
9
8


In [25]:
bound = x.double
bound.__self__, bound.__func__

(<__main__.Number at 0x33fa7d0>, <function __main__.Number.double(self)>)

In [26]:
bound.__self__.base # self of base: x.base

2

In [27]:
bound() # return function

4

In [28]:
def square(arg):       # function normal
    return arg ** 2 

In [29]:
class Sum:
    def __init__(self, val): # Callable instances
        self.val = val
    def __call__(self, arg):
        return self.val + arg

In [30]:
class Product:
    def __init__(self, val): # Bound methods
        self.val = val
    def method(self, arg):
        return self.val * arg

In [31]:
sobject = Sum(2)
pobject = Product(3)
actions = [square, sobject, pobject.method]
for act in actions: # All three called same way
    print(act(5)) # Call any one-ar

25
7
15


In [32]:
actions[-1](5)

15

In [33]:
[act(5) for act in actions]

[25, 7, 15]

In [34]:
 list(map(lambda act: act(5), actions))

[25, 7, 15]

In [35]:
class Negate:
    def __init__(self, val): # Classes are callables too
        self.val = -val # But called for object, not work
    def __repr__(self): # Instance print format
        return str(self.val)

In [36]:
actions = [square, sobject, pobject.method, Negate] # Call a class too
for act in actions:
    print(act(5))

25
7
15
-5


In [37]:
[act(5) for act in actions] # Runs __repr__ not __str__

[25, 7, 15, -5]

In [38]:
table = {act(5): act for act in actions}
table

{25: <function __main__.square(arg)>,
 7: <__main__.Sum at 0x341c1f0>,
 15: <bound method Product.method of <__main__.Product object at 0x0341C250>>,
 -5: __main__.Negate}

# 8.6 Object Factory

In [39]:
# File factory.py
def factory(aClass, *pargs, **kargs): # Varargs tuple, dict
    return aClass(*pargs, **kargs) # Call aClass
class Spam:
    def doit(self, message):
        print(message)
class Person:
    def __init__(self, name, job=None):
        self.name = name
        self.job = job

In [40]:
object1 = factory(Spam) # Make a Spam object
object2 = factory(Person, "Arthur", "King") # Make a Person object
object3 = factory(Person, name='Brian')

In [41]:
object1.doit(99)

99


In [42]:
object2.name, object2.job

('Arthur', 'King')

In [43]:
object3.name, object3.job

('Brian', None)

# 8.7 Multiple Inheritance

In [44]:
class Spam:
    def __init__(self): 
        self.data1 = "food"
X = Spam()
print(X)

In [45]:
# File listinstance.py
class ListInstance:
    def __attrnames(self):
        result = ''
        for attr in sorted(self.__dict__):
            result += '\t%s=%s\n' % (attr, self.__dict__[attr])
        return result
    def __str__(self):
        return '<Instance of %s, address %s:\n%s>' % (
                    self.__class__.__name__, # My class's name
                    id(self), # My address
                    self.__attrnames()) # name=value list
if __name__ == '__main__':
    import testmixin
    testmixin.tester(ListInstance)

<Instance of Sub, address 54691440:
	data1=spam
	data2=eggs
	data3=42
>


In [46]:
# File textmixin0.py
class Super:
    def __init__(self): # Superclass __init__
        self.data1 = 'spam' # Create instance attrs
    def ham(self):
        pass
class Sub(Super, ListInstance): # Mix in ham and a __str__
    def __init__(self): # Listers have access to self
        Super.__init__(self)
        self.data2 = 'eggs' # More instance attrs
        self.data3 = 42
    def spam(self): # Define another method here
        pass
if __name__ == '__main__':
    X = Sub()
    print(X) # __str__

<Instance of Sub, address 54692976:
	data1=spam
	data2=eggs
	data3=42
>


In [47]:
# File testmixin.py
import importlib
def tester(listerclass, sept=False):
    class Super:
        def __init__(self): # Superclass __init__
            self.data1 = 'spam' # Create instance attrs
        def ham(self):
            pass
    class Sub(Super, listerclass): # Mix in ham and a __str__
        def __init__(self): # Listers have access to self
            Super.__init__(self)
            self.data2 = 'eggs' # More instance attrs
            self.data3 = 42
        def spam(self): # Define another method here
            pass
    instance = Sub() # Return instance with lister's __str__
    print(instance) # Run mixed-in __str__ (or via str(x))
    if sept: print('-' * 80)
        
def testByNames(modname, classname, sept=False):
    modobject = importlib.import_module(modname) # Import by namestring
    listerclass = getattr(modobject, classname) # Fetch attr by namestring
    tester(listerclass, sept)
if __name__ == '__main__':
    testByNames('listinstance', 'ListInstance', True) # Test all three here
#     testByNames('listinherited', 'ListInherited', True)
#     testByNames('listtree', 'ListTree', False)

<Instance of Sub, address 54753104:
	data1=spam
	data2=eggs
	data3=42
>
--------------------------------------------------------------------------------


In [48]:
import listinstance
class C(listinstance.ListInstance): pass
x = C()
x.a, x.b, x.c = 1, 2, 3
print(x)

<Instance of C, address 54643152:
	a=1
	b=2
	c=3
>


In [49]:
# File listinherited.py
class ListInherited:
    def __attrnames(self):
        result = ''
        for attr in dir(self):  # Instance dir()
            if attr[:2] == '__' and attr[-2:] == '__':  # Skip internals
                result += '\t%s\n' % attr
            else:
                result += '\t%s=%s\n' % (attr, getattr(self, attr))
        return result

    def __str__(self):
        return '<Instance of %s, address %s:\n%s>' % (
            self.__class__.__name__,  # My class's name
            id(self),  # My address
            self.__attrnames())  # name=value list

if __name__ == '__main__':
    import testmixin
    testmixin.tester(ListInherited)

<Instance of Sub, address 54752016:
	_ListInherited__attrnames=<bound method ListInherited.__attrnames of <testmixin.tester.<locals>.Sub object at 0x03437310>>
	__class__
	__delattr__
	__dict__
	__dir__
	__doc__
	__eq__
	__format__
	__ge__
	__getattribute__
	__gt__
	__hash__
	__init__
	__init_subclass__
	__le__
	__lt__
	__module__
	__ne__
	__new__
	__reduce__
	__reduce_ex__
	__repr__
	__setattr__
	__sizeof__
	__str__
	__subclasshook__
	__weakref__
	data1=spam
	data2=eggs
	data3=42
	ham=<bound method tester.<locals>.Super.ham of <testmixin.tester.<locals>.Sub object at 0x03437310>>
	spam=<bound method tester.<locals>.Sub.spam of <testmixin.tester.<locals>.Sub object at 0x03437310>>
>


In [50]:
# File listtree.py
class ListTree:
    def __attrnames(self, obj, indent):
        spaces = ' ' * (indent + 1)

        result = ''
        for attr in sorted(obj.__dict__):
            if attr.startswith('__') and attr.endswith('__'):
                result += spaces + '{0}\n'.format(attr)
            else:
                result += spaces + '{0}={1}\n'.format(attr, getattr(obj, attr))
        return result

    def __listclass(self, aClass, indent):
        dots = '.' * indent

        if aClass in self.__visited:
            return '\n{0}<Class {1}:, address {2}: (see above)>\n'.format(
            dots,
            aClass.__name__,
            id(aClass))
        else:
            self.__visited[aClass] = True
            here = self.__attrnames(aClass, indent)
            above = ''
            for super in aClass.__bases__:
                above += self.__listclass(super, indent + 4)
            return '\n{0}<Class {1}, address {2}:\n{3}{4}{5}>\n'.format(
                    dots,
                    aClass.__name__,
                    id(aClass),
                    here, above,
                    dots)

    def __str__(self):
        self.__visited = {}

        here = self.__attrnames(self, 0)
        above = self.__listclass(self.__class__, 4)
        return '<Instance of {0}, address {1}:\n{2}{3}>'.format(
            self.__class__.__name__,
            id(self),
            here, above)
if __name__ == '__main__':
    import testmixin
    testmixin.tester(ListTree)

<Instance of Sub, address 54768976:
 _ListTree__visited={}
 data1=spam
 data2=eggs
 data3=42

....<Class Sub, address 54804528:
     __doc__
     __init__
     __module__
     spam=<function tester.<locals>.Sub.spam at 0x0342ADF8>

........<Class Super, address 54721816:
         __dict__
         __doc__
         __init__
         __module__
         __weakref__
         ham=<function tester.<locals>.Super.ham at 0x0342AE88>

............<Class object, address 1679650024:
             __class__
             __delattr__
             __dir__
             __doc__
             __eq__
             __format__
             __ge__
             __getattribute__
             __gt__
             __hash__
             __init__
             __init_subclass__
             __le__
             __lt__
             __ne__
             __new__
             __reduce__
             __reduce_ex__
             __repr__
             __setattr__
             __sizeof__
             __str__
             __subc